In [ ]:
import os
import json
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from tqdm import tqdm


# 2. 논문 파일 경로 리스트 (예: ["paper1.pdf", "paper2.pdf", ...])
pdf_paths = [
    "./메타버스_가상_콘서트_소비자.pdf",
    "./가상 아이돌 이용자 연구.pdf",
    "./BTS와 이세계아이돌 유튜브 콘텐츠 댓글 비교 형용사 사용 패턴과 텍스트마이닝을 통한 정서.pdf",
]

# 3. PDF → 텍스트 분할기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=50
)

# 4. LLM 및 프롬프트 설정
llm = OpenAI(model='gpt-4.1-nano')

prompt = PromptTemplate(
    input_variables=["text"],
    template="""
첨부한 논문에서 다음 세 가지 정보를 키워드(간단명) 형태로 뽑아줘:
1. 분야 (field)
2. 주제 (topics)
3. 사용된 연구방법론 (methodologies)

결과를 JSON으로만 반환해줘. 
예시:
{{
  "field": ["Computer Vision", "..."],
  "topics": ["object detection", "..."],
  "methodologies": ["CNN", "transfer learning", "..."]
}}
"""
)

chain = LLMChain(llm=llm, prompt=prompt)

# 5. 전체 논문 순회하며 처리
results = {}
for path in tqdm(pdf_paths, desc="논문 처리"):
    # (1) 로드
    loader = PyMuPDFLoader(path)
    docs = loader.load()
    # (2) 텍스트 청크로 분할
    chunks = text_splitter.split_documents(docs)
    # (3) 각 청크에 LLM 요청, 결과 수집
    aggregated = {"field": set(), "topics": set(), "methodologies": set()}
    for chunk in chunks:
        resp = chain.run(text=chunk.page_content)
        try:
            data = json.loads(resp)
        except json.JSONDecodeError:
            continue
        for key in aggregated:
            for item in data.get(key, []):
                aggregated[key].add(item.strip())
    # (4) set → list 변환 후 저장
    results[os.path.basename(path)] = {
        "field": list(aggregated["field"]),
        "topics": list(aggregated["topics"]),
        "methodologies": list(aggregated["methodologies"])
    }

# 6. 최종 JSON 출력
print(json.dumps(results, ensure_ascii=False, indent=2))

논문 처리: 100%|██████████| 3/3 [01:12<00:00, 24.33s/it]

{
  "메타버스_가상_콘서트_소비자.pdf": {
    "field": [],
    "topics": [],
    "methodologies": []
  },
  "가상 아이돌 이용자 연구.pdf": {
    "field": [],
    "topics": [],
    "methodologies": []
  },
  "BTS와 이세계아이돌 유튜브 콘텐츠 댓글 비교 형용사 사용 패턴과 텍스트마이닝을 통한 정서.pdf": {
    "field": [],
    "topics": [],
    "methodologies": []
  }
}


In [8]:
sample = docs[:1]  # 첫 청크만
info = extract_chain.run_and_parse(sample)
print(info.dict())

NameError: name 'extract_chain' is not defined